In [1]:
import gzip
import shutil

import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bokeh.io import output_file
from bokeh.plotting import figure, output_file, output_notebook, show, curdoc
from bokeh.models import ColumnDataSource, Dropdown, MultiSelect, Select, CustomJS, MultiLine
from bokeh.layouts import gridplot, row, column
from bokeh.models.tools import HoverTool

In [2]:
output_file("primeiroTeste.html", title="Projeto Super")

In [3]:
def coletorCovid(url, localArquivo):
    response = requests.get(url)
    soup= BeautifulSoup(response.text, "html.parser")     
    for link in soup.select("a[href$='caso_full.csv.gz']"): ## tem que mudar o ' ' do href de acordo com o site e arquivo
        filename = os.path.join(localArquivo,link['href'].split('/')[-1])
        with open(filename, 'wb') as f:
            f.write(requests.get(urljoin(url,link['href'])).content)
            print(f, 'Download Terminado')
    print("DOWNLOAD CONCLUÍDO")

In [4]:
def descompactar(arquivo):
    with gzip.open(arquivo, 'rb') as entrada:
        with open('caso_full.csv', 'wb') as saida:
            shutil.copyfileobj(entrada, saida)

In [5]:
def coletorMobilidade(url, arquivo):
    r = requests.get(url)
    with open(arquivo, 'wb') as code:
        code.write(r.content)

In [6]:
def make_plot(source, x, y, nome_y, titulo, cor):
    p = figure(toolbar_location='below', tools="pan,box_zoom,reset,save",
               border_fill_alpha=0.25,
           title=titulo,
           plot_height=550, plot_width=650,
           x_axis_label='Datas',
           x_axis_type='datetime',
           y_axis_label=nome_y)
    
    p.circle(x, y, line_color=cor, legend_label=titulo, source=source)
    p.line(x, y, line_color=cor, legend_label=titulo, source=source)
    p.legend.location = 'top_left'
    hover = HoverTool(tooltips=[
                    ('Cidade', '@city'),
                    ('Data', '@nova_data'),
                    (nome_y, '@'+y)])
                    
    p.add_tools(hover)
    
    return p

In [7]:
urlCovid = "https://data.brasil.io/dataset/covid19/_meta/list.html"
urlMobility = "https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv?cachebust=94537edba4db1128"
localArquivo = r'C:\Users\DMG70\Desktop\ProjetoSuper\terceiraParte'#local onde deve ser salvo os arquivo

In [8]:
coletorCovid(urlCovid, localArquivo)
descompactar('caso_full.csv.gz')
coletorMobilidade(urlMobility, 'global_mobility.csv')

<_io.BufferedWriter name='C:\\Users\\DMG70\\Desktop\\ProjetoSuper\\terceiraParte\\caso_full.csv.gz'> Download Terminado
DOWNLOAD CONCLUÍDO


In [9]:
df_covid_br = pd.read_csv('caso_full.csv', usecols=[0, 2, 3, 7, 11, 13, 14, 15, 16]).dropna()
df_covid_br['nova_data'] = df_covid_br.date
df_covid_br['date'] = pd.to_datetime(df_covid_br.date)#df_covid_br.date, infer_datetime_format=True)

df_mobilidade = pd.read_csv('global_mobility.csv')
df_mobilidade.rename(columns={'sub_region_2':'city'}, inplace=True)
df_mobilidade['nova_data'] = df_mobilidade.date
df_mobilidade['date'] = pd.to_datetime(df_mobilidade.date, infer_datetime_format=True)

C:\Users\DMG70\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
df_cidade = df_covid_br[df_covid_br.city == 'Manaus']
cds_cidade = ColumnDataSource(df_cidade)

plot1 = make_plot(cds_cidade, 'date', 'last_available_confirmed', 'Quantidade', 'Casos Confirmados', "green")
plot2 = make_plot(cds_cidade, 'date', 'last_available_deaths', 'Quantidade', 'Óbitos Confirmados',"red")

In [11]:
df_mobilidade_cidade = df_mobilidade[df_mobilidade.city == 'Manaus']
cds_mobilidade_cidade = ColumnDataSource(df_mobilidade_cidade)


plot3 = make_plot(cds_mobilidade_cidade, 'date', 'retail_and_recreation_percent_change_from_baseline', 'Porcentagem(%)', 'Mobilidade(%): Varejo e Recreação', 'black')
plot4 = make_plot(cds_mobilidade_cidade, 'date', 'grocery_and_pharmacy_percent_change_from_baseline', 'Porcentagem(%)', 'Mobilidade(%): Mercearia e Farmacia', '#FFFF00')
plot5 = make_plot(cds_mobilidade_cidade, 'date', 'parks_percent_change_from_baseline',  'Porcentagem(%)', 'Mobilidade(%): Parques', '#FF00FF')
plot6 = make_plot(cds_mobilidade_cidade, 'date', 'transit_stations_percent_change_from_baseline', 'Porcentagem(%)', 'Mobilidade(%): Estações de Transporte Público', '#BC8F8F')
plot7 = make_plot(cds_mobilidade_cidade, 'date', 'workplaces_percent_change_from_baseline', 'Porcentagem(%)', 'Mobilidade(%): Locais de Trabalho', '#00FF00')
plot8 = make_plot(cds_mobilidade_cidade, 'date', 'residential_percent_change_from_baseline', 'Porcentagem(%)', 'Mobilidade(%): Moradia', '#8B4513')


In [12]:
df_estado = df_covid_br[df_covid_br.state=='AM']
cidades = list(df_estado.city.unique() )
select_cidade =  Select(options=cidades, value="Manaus", title="Cidade:")

In [13]:
tipo_mob = ['Varejo e Recreação', 'Mercearia e Famácia', 'Parques', 'Estações de Transporte Público', 'Locais de Trabalho', 'Moradia']
select_mobilidade =  Select(options=tipo_mob, value="Varejo e Recreação", title="Mobilidade:")

In [14]:
layout = column(row(select_cidade), row(plot1, plot2),row(select_mobilidade), row(plot3, plot4), row(plot5, plot6), row(plot7, plot8))

In [15]:
show(layout)